##### J-POP 年代別　歌詞抽出

In [2]:
import pandas as pd
import numpy as np
import requests
import re
import urllib
import time
from bs4 import BeautifulSoup

#### STEP1:ダウンロード数ランキング

In [12]:
years = [year for year in range(2010,2020)]

In [13]:
for (i,year) in enumerate(years):
    url = 'https://nendai-ryuukou.com/%s/download.html'%year

    r = requests.get(url)
    content_type_encoding = r.encoding if r.encoding != 'ISO-8859-1' else None
    soup = BeautifulSoup(r.content, 'html.parser', from_encoding=content_type_encoding)

    text = soup.text

    idx_s = text.find('アーティスト')
    text = text[idx_s:]
    idx_e = text.find('google')
    text = text[:idx_e]

    pattern = '\d\\d位'
    text = re.sub(pattern, '',text)

    text_split = text.split('\n')
    extract = [t for t in text_split if t != ''][1:-1]

    n = int(len(extract)/2)
    songs = extract[::2]
    artists = extract[1::2]
    order = [i+1 for i in range(n)]
    year_list = [year for i in range(n)]


    df_extract = pd.DataFrame({'year':year_list,'order':order,'artists':artists,'songs':songs})
    if i == 0:
        df_extract_tol = df_extract
    else:
        df_extract_tol = pd.concat([df_extract_tol,df_extract])
    time.sleep(1)

In [14]:
df_extract_tol.head()

,year,order,artists,songs
0,2010,1,木村カエラ,Butterfly
1,2010,2,Superfly,タマシイレボリューション
2,2010,3,AKB48,ポニーテールとシュシュ
3,2010,4,斉藤 和義,ずっと好きだった
4,2010,5,AKB48,ヘビーローテーション


In [15]:
df_extract_tol = df_extract_tol[df_extract_tol.order<=20]

#### STEP2：歌詞の抽出

In [16]:
df = df_extract_tol.copy()
print(df.shape)

(200, 4)


In [78]:
lyrics = []
for count,(song, artist) in enumerate(zip(df.songs, df.artists)):
    lyric = None
    encode = urllib.parse.quote('%s'%song)
    s_url = 'https://www.uta-net.com/search/?Aselect=2&Keyword=%s&Bselect=4'%encode
    html = requests.get(s_url).text
    splits = html.split('class="side td1"')
    if len(splits) > 1:#検索に引っかからないパターンもある．
        url_front = 'https://www.uta-net.com'
        for split in splits[1:]:
            l_url = split.split('">')[0].replace('><a href="','')
            l_url = url_front + l_url
            l_text = requests.get(l_url).text
            if artist in l_text.split('artist_name')[1].split('\r')[0]:
                pattern = 'itemprop="text">'

                lyric = re.sub('<br />',' ',l_text.split(pattern)[1])
                idx = lyric.find('</div>')
                lyric = lyric[:idx]
                print(artist,lyric[:10])
                lyrics.append(lyric)
            time.sleep(1)
    
    if lyric == None:
        lyrics.append('')
    time.sleep(1)
    
    if count%10 == 0:
        time.sleep(2)
        print('sleep...ready?')

木村カエラ Butterfly　
sleep...ready?
Superfly スタンドアップ!モン
AKB48 カレンダーより早く 
AKB48 I want you
FUNKY MONKEY BABYS こぼれ落ちるその涙が
坂本冬美 朝露が招く　光を浴び
木村カエラ ring a din
EXILE いつでも人は悲しみを
sleep...ready?
moumoon キラキラ　陽差しを浴
FUNKY MONKEY BABYS あと一粒の涙で　ひと
KARA la la la l
Superfly 果てまで続く　坂道を
FUNKY MONKEY BABYS あなたが笑ってて　あ
FUNKY MONKEY BABYS 最寄り駅の改札抜けれ
sleep...ready?
KARA la la la l
KARA Jumpin' Ju
Mr.Children かぞえうた さぁ な
少女時代 Listen Boy
コブクロ 真冬の海辺に映った　
AKB48 I want you
AKB48 Na Na Na  
AKB48 太陽が 昨日より 眩
KARA ネテモサメテモスキサ
sleep...ready?
少女時代 TOKYO, SEO
植村花菜 小3の頃からなぜだか
AKB48 In your po
ナオト・インティライミ 今キミを　今のキミを
EXILE いつでも君の為にI 
EXILE いつでも君の為にI 
AKB48 カレンダーより早く 
sleep...ready?
斉藤和義 地球儀を回して世界1
宇多田ヒカル 開いたばかりの花が散
安室奈美恵 昔からラブストーリー
AKB48 桜の歌が街に流れ あ
EXILE So, Rising
GReeeeN いつのまに　忘れてた
ゴールデンボンバー 女々しくて女々しくて
sleep...ready?
Ms.OOJA こんな世界で一人きり
Superfly どれくらい感謝したっ
ゆず 誰にも見せない泪があ
家入レオ 夜明けを待ち続けた　
ナオト・インティライミ 君に会いたかった　た
AKB48 こんなに君を 好きで
きゃりーぱみゅぱみゅ 鮮やかに　恋して　に
sleep...ready?
AKB48 あなたのことが好きな
ナオト・インティライミ なぜだろう　今夜は　
ゴールデンボンバー 女々しくて女々しくて
ケラケラ 

In [110]:
df_lyrics = pd.DataFrame({'lyrics':lyrics})

In [111]:
#df_lyrics.to_csv('df_lyrics.csv',index=None)

In [17]:
df_lyrics = pd.read_csv('df_lyrics.csv')

In [18]:
df[df.artists=='EXILE']

,year,order,artists,songs
9,2010,10,EXILE,もっと強く
16,2011,17,EXILE,I Wish For You
7,2012,8,EXILE,Rising Sun


exile ,いつでも君の為にI...のやつが被っている

In [19]:
df_lyrics[df_lyrics.lyrics.str.contains('いつでも君の為にI') == True]

,lyrics
36,いつでも君の為にI wish 幸せを願う事こそが愛 夢を見れば恋をすれば 誰にでも悩める日が...
37,いつでも君の為にI wish 幸せを願う事こそが愛 夢を見れば恋をすれば 誰にでも悩める日が...


In [20]:
df_lyrics_drop = df_lyrics.drop(36,axis=0).copy()

In [21]:
df['lyrics'] = df_lyrics_drop['lyrics'].to_list()

In [23]:
df.to_csv('j_pop_songs.csv',index=None)

In [2]:
df = pd.read_csv('j_pop_songs.csv')

In [3]:
df.head()

,year,order,artists,songs,lyrics
0,2010,1,木村カエラ,Butterfly,Butterfly 今日は今までの どんな時より 素晴らしい 赤い糸でむすばれてく 光の輪の...
1,2010,2,Superfly,タマシイレボリューション,スタンドアップ!モンスター 頂上へ 道なき道を 切り開く時 スタンドアップ!ファイター とん...
2,2010,3,AKB48,ポニーテールとシュシュ,カレンダーより早く シャツの袖口まくって 太陽が近づく気配 僕の腕から衣替え 青い海 波打...
3,2010,4,斉藤 和義,ずっと好きだった,NaN
4,2010,5,AKB48,ヘビーローテーション,I want you! I need you! I love you! 頭の中 ガンガン鳴っ...
